## إعداد متغيرات البيئة

In [ ]:
from IPython.display import display, clear_output
import ipywidgets as widgets
import os

api_id_input = widgets.Text(
    placeholder='ادخل API_ID',
    description='API_ID:',
    value=''
)
api_hash_input = widgets.Text(
    placeholder='ادخل API_HASH',
    description='API_HASH:',
    value=''
)
session_input = widgets.Textarea(
    placeholder='ادخل SESSION',
    description='SESSION:',
    value='',
    layout=widgets.Layout(width='80%', height='100px')
)
bot_token_input = widgets.Text(
    placeholder='ادخل BOT_TOKEN',
    description='BOT_TOKEN:',
    value=''
)
save_button = widgets.Button(description='حفظ المتغيرات', button_style='success')
output = widgets.Output()

def save_variables(btn):
    with output:
        clear_output()
        api_id = api_id_input.value.strip()
        api_hash = api_hash_input.value.strip()
        session = session_input.value.strip()
        bot_token = bot_token_input.value.strip()
        if not all([api_id, api_hash, session, bot_token]):
            print('يرجى ملء جميع الحقول')
            return
        os.makedirs('SiFThon', exist_ok=True)
        env_path = os.path.join('SiFThon', '.env')
        with open(env_path, 'w', encoding='utf-8') as f:
            f.write(f'API_ID={api_id}\nAPI_HASH={api_hash}\nSESSION={session}\nBOT_TOKEN={bot_token}\n')
        print(f'✅ تم حفظ المتغيرات في {env_path}')

save_button.on_click(save_variables)
display(api_id_input, api_hash_input, session_input, bot_token_input, save_button, output)

## تنزيل وتشغيل السكربت باستخدام python -m SiFThon

In [ ]:
import requests, zipfile, subprocess, urllib3
from IPython.display import display, clear_output
import ipywidgets as widgets
import os, sys, textwrap

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

url_input = widgets.Text(placeholder='ادخل رابط ملف ZIP', description='رابط ZIP:', layout=widgets.Layout(width='80%'))
download_button = widgets.Button(description='تحميل وتشغيل (python -m SiFThon)', button_style='info')
process_output = widgets.Output()

def download_and_run(btn):
    with process_output:
        clear_output()
        url = url_input.value.strip()
        if not url:
            print('يرجى ادخال رابط ملف ZIP')
            return
        try:
            print('⏳ جاري التحميل...')
            response = requests.get(url, stream=True, verify=False, timeout=30)
            response.raise_for_status()
            zip_filename = 'downloaded_file.zip'
            with open(zip_filename,'wb') as f:
                for chunk in response.iter_content(chunk_size=8192): f.write(chunk)
            print('✅ تم التحميل بنجاح')
            print('⏳ جاري فك الضغط...')
            with zipfile.ZipFile(zip_filename,'r') as zip_ref: zip_ref.extractall('.')
            print('✅ تم فك الضغط بنجاح')
            if not os.path.exists('SiFThon'):
                print('❌ لم يتم العثور على مجلد SiFThon بعد فك الضغط')
                return
            env_path = os.path.join('SiFThon', '.env')
            if not os.path.exists(env_path):
                print('❌ لم يتم العثور على ملف .env داخل مجلد SiFThon')
                print('يرجى ملء المتغيرات في الخلية المخصصة أولاً')
                return
            print('✅ تم العثور على مجلد SiFThon وملف .env')
            # تثبيت المكتبات إذا موجود requirements.txt
            req_path = os.path.join('SiFThon','requirements.txt')
            if os.path.exists(req_path):
                print('⏳ جاري تثبيت المكتبات من requirements.txt ...')
                result = subprocess.run([sys.executable, '-m', 'pip', 'install', '-r', req_path], capture_output=True, text=True)
                if result.returncode==0: print('✅ تم تثبيت المكتبات بنجاح')
                else: print('❌ خطأ في تثبيت المكتبات:',result.stderr)
            else:
                print('⚠️ لم يتم العثور على ملف requirements.txt داخل SiFThon')
            # إنشاء __main__.py مؤقت إذا لم يوجد
            main_stub = os.path.join('SiFThon','__main__.py')
            created_main_stub = False
            if not os.path.exists(main_stub):
                stub_content = textwrap.dedent('''from .main import *''')
                with open(main_stub,'w',encoding='utf-8') as f: f.write(stub_content)
                created_main_stub = True
                print('ℹ️ تم إنشاء __main__.py مؤقت لتشغيل python -m SiFThon')
            print('⏳ جاري تشغيل python -m SiFThon ...')
            cwd_parent = os.path.abspath('.')
            result = subprocess.run([sys.executable, '-m', 'SiFThon'], cwd=cwd_parent, capture_output=True, text=True)
            print('📋 مخرجات السكربت:')
            print(result.stdout or '(لا توجد مخرجات stdout)')
            if result.stderr: print('❌ اخطاء:',result.stderr)
            # حذف __main__.py المؤقت بعد التشغيل
            if created_main_stub:
                try: os.remove(main_stub); print('ℹ️ تم حذف __main__.py المؤقت')
                except: pass
            print('✅ اكتملت العملية')
        except Exception as e:
            print(f'❌ حدث خطأ: {e}')

download_button.on_click(download_and_run)
display(url_input, download_button, process_output)